<a href="https://colab.research.google.com/github/BillXu21/culvert-identification/blob/main/EfficientNet_HRDEM_(98%25).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
 #!pip install rasterio

In [ ]:
import sys
print(sys.version)

3.10.12 (main, Jun 11 2023, 05:26:28) [GCC 11.4.0]


In [ ]:
import tensorflow as tf
print(tf.__version__)

2.13.0


In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

from skimage import io
import glob

from sklearn.model_selection import train_test_split
# import tensorflow as tf

import tensorflow as tf

import numpy as np
import pandas as pd


#from tensorflow.keras.callbacks import TensorBoard
from tensorflow.compat.v1.keras.models import Sequential
from tensorflow.compat.v1.keras.layers import BatchNormalization
from tensorflow.compat.v1.keras.layers import Dense, Dropout, Flatten, Activation
from tensorflow.compat.v1.keras.layers import Convolution2D, MaxPooling2D, AveragePooling2D
from tensorflow.compat.v1.keras.optimizers import Adadelta, Adam
from tensorflow.compat.v1.keras.callbacks import ModelCheckpoint
from tensorflow import keras

from sklearn.preprocessing import MinMaxScaler

#from tensorflow.keras import callbacks
import matplotlib.pyplot as plt

tf.compat.v1.disable_eager_execution()

In [ ]:
path='/content/drive/MyDrive/WFBBDEM/'   #DEM

# using one watershed to train model, 3 of them to test
pathTrain='WFBBDEM_Train/'
pathTest1='WFBBDEM_Test/'

In [ ]:
# # read all images
# import rasterio as rio
# def read_img(path):
#     cate=[path+x for x in os.listdir(path) if os.path.isdir(path+x)] # get F,T folder
#     imgs=[]
#     labels=[]
#     imgs_name=[]
#     cate = cate[:2]
#     for idx,folder in enumerate(cate): # idx-> 0:F; 1:T; folder-> F,T
#         for im in glob.glob(folder+"/*.tif"):
#             im = im.replace('\\','/')
# #            print('reading the images:%s'%(im))

#             img_name=os.path.basename(im)
#             img_name=os.path.splitext(img_name)[0] # get file name

#             #img=io.imread(im)
#             with rio.open(im) as i:
#               img = i.read()
#             # Normalize the dataset-MaxMin
#             img = np.squeeze(img)
#             scaler = MinMaxScaler(feature_range=(0, 1))
#             img = scaler.fit_transform(img)

#             imgs.append(img)
#             labels.append(idx)
#             imgs_name.append(img_name) # image name

#     return np.asarray(imgs,np.float32),np.asarray(labels,np.int32), np.asarray(imgs_name)

# data,label,name=read_img(path+pathTrain)

In [ ]:
#### Test data-3 watersheds
#dataTst1, labelTst1, nameTst1 =read_img(path+pathTest1)

In [ ]:
# import pickle
# with open('/content/drive/MyDrive/WFBBDEM/CNN_Train/data','wb') as fin: pickle.dump(data,fin)
# with open('/content/drive/MyDrive/WFBBDEM/CNN_Train/label','wb') as fin: pickle.dump(label,fin)
# with open('/content/drive/MyDrive/WFBBDEM/CNN_Train/name','wb') as fin: pickle.dump(name,fin)

In [ ]:
import pickle
with open('/content/drive/MyDrive/WFBBDEM/CNN_Train/data','rb') as fin: data = pickle.load(fin)
with open('/content/drive/MyDrive/WFBBDEM/CNN_Train/label','rb') as fin: label = pickle.load(fin)
with open('/content/drive/MyDrive/WFBBDEM/CNN_Train/name','rb') as fin: name = pickle.load(fin)

In [ ]:
#data=np.expand_dims(data,3) # add channel dimension, 4D
data = np.repeat(data[..., np.newaxis], 3, -1)

labelN=label.shape[0]


label_name=np.empty((labelN,2),dtype=int) # label + name
for i in range(labelN):
    label_name[i,0]=label[i] # label
    if (name[i].isdigit()):
        name[i]=name[i]
    else:
        name[i]=name[i][1:] # only keep number, delete first letter
    name[i] = name[i].split()[0]
    label_name[i,1]=name[i]

In [ ]:
# dataTst1=np.expand_dims(dataTst1,3)
# ######Test1
# labelN_Tst1=labelTst1.shape[0]

# label_nameTst1=np.empty((labelN_Tst1,2),dtype=int) # label + name
# for i in range(labelN_Tst1):
#     label_nameTst1[i,0]=labelTst1[i] # label
#     if (nameTst1[i].isdigit()):
#         nameTst1[i]=nameTst1[i]
#     else:
#         nameTst1[i]=nameTst1[i][1:] # only keep number, delete first letter
#     label_nameTst1[i,1]=nameTst1[i]

In [ ]:
# split data into train and test groups
train_data,test_data,train_label,test_label = train_test_split(data,label_name,test_size=0.2,stratify=label)

In [ ]:
# creat CNN model
def createCNN():
    model = Sequential()


    model.add(Convolution2D(128, 3, 3, padding="same", input_shape=(100, 100, 1)))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(AveragePooling2D(pool_size=(2,2)))
#    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.30))


    model.add(Convolution2D(256, (3,3), padding="same"))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(AveragePooling2D(pool_size=(2,2)))
#    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.30))


    model.add(Convolution2D(512, (5,5), padding="same"))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(AveragePooling2D(pool_size=(2,2)))
#    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.30))


    model.add(Convolution2D(1024, (5,5), padding="same"))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(AveragePooling2D(pool_size=(2,2)))
#    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.30))

    model.add(Flatten())

    model.add(Dense(512, activation = 'relu'))
    model.add(Dropout(0.5))

    model.add(Dense(2, activation = 'softmax'))

    model.summary()

    return model

In [ ]:
LEARNING_RATE = 1e-6
BATCH_SIZE = 64
EPOCHS = 10


model = tf.keras.applications.EfficientNetV2M(
    weights='/content/drive/MyDrive/WFBBDEM/Models/ModelDEM2022EfficientNetM.h5',
    include_top=True,
    input_tensor=None,
    input_shape=(100,100,3),
    pooling="max",
    classes = 2,
    classifier_activation="softmax"
)

# import tensorflow as tf
# from tensorflow import keras

# classification_head = keras.Sequential(
#     [
#         keras.layers.Dense(2, activation='softmax')
#     ],
#     name='classification_head'
# )

# #x = neck_branch(model.output)
# output_a = classification_head(model.output)
# model = keras.Model(model.inputs, [output_a])

model.compile(loss='sparse_categorical_crossentropy',
              optimizer=Adam(lr=LEARNING_RATE),
              metrics=['accuracy'])

/usr/local/lib/python3.10/dist-packages/keras/src/optimizers/legacy/adam.py:118: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


In [ ]:
#model.summary()

In [ ]:
import gc
gc.collect()

0

In [ ]:
OUT_DIR = "/content/drive/MyDrive/WFBBDEM/Models"
checkpoint = ModelCheckpoint(os.path.join(OUT_DIR, 'ModelDEM2022EfficientNetM1.h5'),  # model filename
                              monitor='val_accuracy', # quantity to monitor
                              verbose=1, # verbosity - 0 or 1
                              save_best_only= True, # The latest best model will not be overwritten
                              save_weights_only=True, # save model, not only weights
                              mode='auto') # The decision to overwrite model is made
                                            # automatically depending on the quantity to monitor

model_details = model.fit(train_data, train_label[:,0],
                          batch_size = BATCH_SIZE,
                          epochs = EPOCHS,
                          validation_split=0.2,
                          callbacks=[checkpoint],
                          verbose=1)


Train on 2324 samples, validate on 582 samples
Epoch 1/10
2324/2324 [==============================] - ETA: 0s - loss: 0.1859 - accuracy: 0.9600

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training_v1.py:2335: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates = self.state_updates



Epoch 1: val_accuracy improved from -inf to 0.97079, saving model to /content/drive/MyDrive/WFBBDEM/Models/ModelDEM2022EfficientNetM1.h5
2324/2324 [==============================] - 614s 264ms/sample - loss: 0.1859 - accuracy: 0.9600 - val_loss: 0.1412 - val_accuracy: 0.9708
Epoch 2/10
2324/2324 [==============================] - ETA: 0s - loss: 0.1687 - accuracy: 0.9617
Epoch 2: val_accuracy did not improve from 0.97079
2324/2324 [==============================] - 25s 11ms/sample - loss: 0.1687 - accuracy: 0.9617 - val_loss: 0.1331 - val_accuracy: 0.9691
Epoch 3/10
1536/2324 [==================>...........] - ETA: 7s - loss: 0.1575 - accuracy: 0.9616

In [ ]:
scroe, accuracy = model.evaluate(test_data, test_label[:,0], batch_size=BATCH_SIZE)
print('EfficientNet_Test: loss:', scroe, 'accuracy:', accuracy)


In [ ]:
# Predict
pred_label = model.predict(test_data)
print(pred_label)
pred_label = np.argmax(pred_label, axis=1)


for t in range(pred_label.shape[0]):
    NameWrong=[]
    labelPred=[]
    if (pred_label[t]!=test_label[t,0]):
        # print (test_label[t,0], test_label[t,1], pred_label[t])
        NameWrong.append(test_label[t,1])
        labelPred.append(pred_label[t])


In [ ]:
#### plot confusion matrix
def plot_confusion_matrix(confusion_mat):
    plt.imshow(confusion_mat,interpolation='nearest',cmap=plt.cm.Wistia)
    plt.title('CNN Confusion Matrix',fontsize=20)
    plt.colorbar()
    tick_marks=np.arange(2) # class number
    plt.xticks(tick_marks,tick_marks,fontsize=16)
    plt.yticks(tick_marks,tick_marks,fontsize=16)
    plt.ylabel('True Label',fontsize=16)
    plt.xlabel('Predicted Label',fontsize=16)
    for i in range(len(confusion_mat)):    #row
        for j in range(len(confusion_mat[i])):    #col
            plt.text(j, i, confusion_mat[i][j],fontsize=16) # images number of each part
    plt.show()



confusion_matrix = tf.math.confusion_matrix(labels=test_label[:,0],predictions=pred_label, num_classes=2, dtype=tf.int32, name=None, weights=None)


In [ ]:
sess=tf.compat.v1.Session()

#with tf.compat.v1.Session(graph=g) as sess:
confusion_matrix = sess.run(confusion_matrix)
plot_confusion_matrix(confusion_matrix)



In [ ]:
######################### learning curve

def plot_learning_curves(history):
    df=pd.DataFrame(history.history,index=np.arange(0, EPOCHS).astype(dtype=np.str))
    df.plot(use_index=True,figsize=(8, 5))
    plt.grid(True)

    #plt.gca().set_ylim(0, 1)
    plt.show()

plot_learning_curves(model_details)

print ('Accuracy:', '{:.4f}'.format(accuracy), 'Batch Size:', BATCH_SIZE,'Learning Rate:', LEARNING_RATE, 'Epochs:', EPOCHS)


In [ ]:
# ########################################### Test

# ################Test1

# scroeTst1, accuracyTst1 = model.evaluate(dataTst1,label_nameTst1[:,0], batch_size=BATCH_SIZE)
# print('CNN_Test_Test1: loss:', scroeTst1, 'accuracy:', accuracyTst1)

# # Predict
# pred_labelTst1 = model.predict(dataTst1)
# pred_labelTst1 = np.argmax(pred_labelTst1, axis=1)


# for t1 in range(pred_labelTst1.shape[0]):
#     NameWrongTst1=[]
#     labelPredTst1=[]
#     if (pred_labelTst1[t1]!=label_nameTst1[t1,0]):
#         NameWrongTst1.append(label_nameTst1[t1,1])
#         labelPredTst1.append(pred_labelTst1[t1])

# # np.save("D:/CNNtest2022/ResultName/NameWrongTst1.npy",NameWrongTst1,save_format='h5') # names of wrongly classified images
# # np.save("D:/CNNtest2022/ResultName/LabelPredTst1.npy",labelPredTst1,save_format='h5')



# # Confusion matrix
# confusion_matrix_Tst1 = tf.math.confusion_matrix(labels=label_nameTst1[:,0],predictions=pred_labelTst1, num_classes=None, dtype=tf.int32, name=None, weights=None)
# # sess=tf.compat.v1.Session()
# sess_Tst1=tf.compat.v1.Session()

# #with tf.compat.v1.Session(graph=g) as sess:
# confusion_matrix_Tst1 = sess_Tst1.run(confusion_matrix_Tst1)
# plot_confusion_matrix(confusion_matrix_Tst1)
